In [1]:
a=4
a


4

In [2]:
#!/usr/bin/python3
from sys import argv
import numpy as np
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt
import os 
import re

def read_sam_chunks(file_input_sam_address, map_qual_tresh):
    file = open(file_input_sam_address,'r');
    pos_list= []; ref_name_list = []
    read_name_chunk_list=[]
    previous_read_name_root = ""
    read_name_root_list=[]
    for line in file:
        line_strip = line.strip()  
        if line_strip and not line_strip.startswith("@"):
            line_split = line_strip.split("\t")
            read_name_chunk, flag_str, ref_name, pos, map_qual = line_split
#             flag_binary=str(bin(int(flag_str)))[2:].zfill(17)
#             not_primary= int(flag_binary[8])  # 8 =  17- log2(256) -1 
#             supplemntry= int(flag_binary[5]) # 5 =  17- log2(2048) -1     
            #print(flag_str, not_primary, supplemntry)
            if ref_name != '*': # mapped 
                read_name_root_splits = read_name_chunk.split("_")[:-2]
                read_name_root="_".join(read_name_root_splits)
                if read_name_root != previous_read_name_root:   #new_record
                    pos_list.append([pos])
                    ref_name_list.append([ref_name])
                    read_name_chunk_part = int(read_name_chunk.split("_")[-1]) # 0/1/2/3/4, id of every 3kb chunk
                    read_name_chunk_list.append([read_name_chunk_part])
                    read_name_root_list.append(read_name_root)          
                else:
                    pos_list[-1].append(pos)
                    ref_name_list[-1].append(ref_name)
                    read_name_chunk_part = int(read_name_chunk.split("_")[-1]) # 0/1/2/3/4, id of every 3kb chunk
                    read_name_chunk_list[-1].append(read_name_chunk_part)
                previous_read_name_root = read_name_root         
    #             if int(map_qual) > map_qual_tresh and not_primary==0 and supplemntry == 0: # and 
    #                 read_name_chunk_list.append(read_name_chunk)
    #                 ref_name_list.append(ref_name)
    #                 pos_list.append(int(pos))
    return (read_name_chunk_list, read_name_root_list, ref_name_list, pos_list)


def read_fai(file_input_fai_address):
    file = open(file_input_fai_address,'r');
    record_name_list= []
    record_length_list=[]
    for line in file:
        line_strip = line.strip()  
        if line_strip and not line_strip.startswith("@"):
            line_split = line_strip.split("\t")
            record_name, record_length = line_split[:2]
            record_name_list.append(record_name)
            record_length_list.append(int(record_length))
    return (record_name_list, record_length_list)


def read_sam_cigar(file_input_sam_address, map_qual_tresh):
    file = open(file_input_sam_address,'r');
    pos_list = []; ref_name_list = []    
    read_name_list =[];     num_matched_base_list = []
    previous_read_name = ""
    for line in file:
        line_strip = line.strip()  
        if line_strip and not line_strip.startswith("@"):
            line_split = line_strip.split("\t")
            read_name, flag_str, ref_name, pos, map_qual, cigar_string = line_split
#             flag_binary=str(bin(int(flag_str)))[2:].zfill(17)
#             not_primary= int(flag_binary[8])  # 8 =  17- log2(256) -1 
#             supplemntry= int(flag_binary[5]) # 5 =  17- log2(2048) -1     
            #print(flag_str, not_primary, supplemntry)
            if int(map_qual) > map_qual_tresh and ref_name != '*':
                matches_list_read = re.findall(r'(\d+)([A-Z]{1})', cigar_string) # s = '6M1I69M1D34M'
                num_matched_base =0
                for (length_str, cigartype) in matches_list_read:
                    if cigartype =='M':
                        num_matched_base += int(length_str)

                if read_name != previous_read_name:   #new_record
                    pos_list.append([int(pos)])
                    ref_name_list.append([ref_name])
                    num_matched_base_list.append([num_matched_base])    
                    read_name_list.append(read_name)
                else:
                    pos_list[-1].append(int(pos))
                    ref_name_list[-1].append(ref_name)
                    num_matched_base_list[-1].append(num_matched_base)
                previous_read_name = read_name
    return (read_name_list, num_matched_base_list, ref_name_list, pos_list)


def find_child_reads_candidate(read_name_list, ref_name_list, num_matched_base_list):
    
    read_name_candidate = []
    for i in range(len(read_name_list)):
        ref_names = ref_name_list[i]
        read_name = read_name_list[i]
        if len(set(ref_names)) == 2 : # each child read should  be mapped on both haps (parents reads)

            pos_ = pos_list[i] 
            num_matched_base = num_matched_base_list[i]
            hap0 = record_name_list[0] # parent read correspond to a haplotype
            hap1 = record_name_list[1] # parent read correspond to a haplotype
            #indeceis of the child read, that mapped onto the hap 0 
            mapped_hap0= [i for i in range(len(ref_names)) if ref_names[i] == hap0]
            #indeceis of thsoe part of child read, that mapped onto the hap 1 
            mapped_hap1= [i for i in range(len(ref_names)) if ref_names[i] == hap1]
            num_matched_base_hap0 = [num_matched_base[i] for i in mapped_hap0]
            num_matched_base_hap1 = [num_matched_base[i] for i in mapped_hap1]

            if sum(num_matched_base_hap0) > 3*chunk_size*0.95 and sum(num_matched_base_hap1) > 3*chunk_size*0.95 :     
                read_name_candidate.append(read_name)
                # more resitercted would be better  could be  overlapped 
                #print("*** read", i ) #  read_name_list[i]
                #print(num_matched_base_hap0, num_matched_base_hap0, pos_)
    #print("there are",len(read_name_candidate), "child candidate reads with num matched enough. ")
    return read_name_candidate

    
def extract_readname_chop_map(file_input_project_address, chrom, id_patch, code_chop):
    #print(file_input_project_address)

    fq_all = file_input_project_address+"HG002_"+str(chrom)+".fastq"
    # ! ls {fq_all}
    i=1
    fq_candidate = file_input_project_address+"HG002_"+str(chrom)+"_candidate_"+str(id_patch)+".fastq"

    ! echo -n "" >  {fq_candidate}
    for read_name in read_name_candidate:
        #print(read_name)
        !sed -n $(grep -n  {read_name} {fq_all} | tr ':' '\t' | cut -f 1),$(($(grep -n  {read_name} {fq_all} | tr ':' '\t' | cut -f 1)+3))p  {fq_all} >>  {fq_candidate}
    #! cat {fq_candidate} | wc -l 
    fq_candidate_chop = file_input_project_address+"HG002_"+str(chrom)+"_candidate_chop_"+str(id_patch)+".fastq"
    chop_length=chunk_size
    ! python3 {code_chop}  {fq_candidate}  {fq_candidate_chop} {chop_length}   >out   2>&1 
    #! wc -l  {fq_candidate_chop} 

    file_input_chop_sam_address = file_input_project_address + "HG002_HG004_"+str(chrom)+"_"+str(id_patch)+"_chop.sam" 
    file_input_chop_part_sam_address = file_input_project_address + "HG002_HG004_"+str(chrom)+"_"+str(id_patch)+"_chop_5part.sam" 
    error_file = file_input_project_address+"error_file.log"
    !minimap2 -ax map-pb -t {threads} {file_input_fa_address} {fq_candidate_chop}  > {file_input_chop_sam_address} 2> error_file
    # blasr HG002_${chr}.fastq  HG004_${chr}_HP1_HP2_${i}.fa -m 3 > ${i}.blasr

    !cut -f 1-5  {file_input_chop_sam_address} > {file_input_chop_part_sam_address}
    # !cat {file_input_chop_part_sam_address} | wc -l 
      
    return (file_input_chop_sam_address, file_input_chop_part_sam_address)





In [3]:

def check_chunks2(read_name_chunk_list, read_name_root_list, ref_name_list, pos_list, file_input_chop_sam_address, file_input_fa_address) :

    found_any = False
    for read_idx in range(len(ref_name_list)): # 
        ref_names = ref_name_list[read_idx] 
        read_name_chunk = read_name_chunk_list[read_idx] # child root name
        ref_names_unique = list(set(ref_names))
        num_ref_names_unique = len(ref_names_unique)
        least_condition_met = True
        if num_ref_names_unique ==1:
            continue 
            # the rest of code is not considred
        if num_ref_names_unique <1 or num_ref_names_unique >2:
            print("issue not any ref ",ref_names_unique)
            continue         #for the rest num_ref_names_unique ==2:

        assert set(record_name_list) == set(ref_names_unique) # but the order might be /is different
        # record_length_list has same order as record_name_list
        hap0 = record_name_list[0] # parent read correspond to a haplotype
        hap1 = record_name_list[1] # parent read correspond to a haplotype
        # expect to have the child's read chunkgs in order [0,1,2,3,4 ] each 3kb
        #indeceis of thsoe part of child read, that mapped onto the hap 0 
        chunk_mapped_hap0 = [i for i in range(len(ref_names)) if ref_names[i] == hap0]
        #indeceis of thsoe part of child read, that mapped onto the hap 1 
        chunk_mapped_hap1 = [i for i in range(len(ref_names)) if ref_names[i] == hap1]

        #if min(indecis_0) != indecis_0[0]: print(indecis_0)
        # to remove such cases indecis_0= [0, 2, 3, 4] recombination is an ecchange of chr arm
        if  len(chunk_mapped_hap0) > 1 and len(chunk_mapped_hap0) != chunk_mapped_hap0[-1]-chunk_mapped_hap0[0]+1:
            least_condition_met= False  # to make sure the consuquence 
        if  len(chunk_mapped_hap1) > 1 and len(chunk_mapped_hap1) != chunk_mapped_hap1[-1]-chunk_mapped_hap1[0]+1:
            least_condition_met= False 
        # to remvoe   [0]  [1]
        if  len(chunk_mapped_hap0) == 1  and len(chunk_mapped_hap0) == 1 : 
            least_condition_met= False            
        # to remove     indecis_0= [0, 1]  indecis_1= [3]
        #if chunk_mapped_hap1[-1] >10000: 
        #    condition_met= False            
        # tekrari bashad ?  [1,1,2]
        pos_all = [int(pos) for pos in  pos_list[read_idx]]
        pos_0 = [pos_all[i] for i in chunk_mapped_hap0]
        pos_1 = [pos_all[i] for i in chunk_mapped_hap1]
        # chunk_mapped_hap0 is the indecis of the list
        # read_name_chunk_0 is the id of chunk (in the sam file )
        read_name_chunk_0 = [read_name_chunk[i] for i in chunk_mapped_hap0]
        read_name_chunk_1 = [read_name_chunk[i] for i in chunk_mapped_hap1]


#         ## for removing this pos_0=[1, 1308] since the next chunk should map after  ~ 3kb
#         if len(pos_0)>1 :
#             diff_abs_pos_0= [ abs(pos_0[i]-pos_0[i-1]) for i in range(1,len(pos_0))]
#             if min(diff_abs_pos_0) < chunk_size * 0.8: least_condition_met= False    
#         if len(pos_1)>1 : 
#             diff_abs_pos_1= [ abs(pos_1[i]-pos_1[i-1]) for i in range(1,len(pos_1))]
#             if min(diff_abs_pos_1) < chunk_size * 0.8: least_condition_met= False    
#         ## to remove this pos_0= [92, 8261]    mapped far from              
#         if len(pos_0)>1 :
#             diff_abs_pos_0 = [ abs(pos_0[i]-pos_0[i-1]) for i in range(1,len(pos_0))]
#             if max(diff_abs_pos_0) > chunk_size * 1.5: least_condition_met= False    
#         if len(pos_1)>1 : 
#             diff_abs_pos_1= [ abs(pos_1[i]-pos_1[i-1]) for i in range(1,len(pos_1))]
#             if max(diff_abs_pos_1) > chunk_size * 1.5: least_condition_met= False    

#         ##to remove these  [1, 7393, 1388, 4373] a chunk is mapped somwhere else 
#         if len(pos_0)>2 :
#             diff_pos_0= [ pos_0[i]-pos_0[i-1] for i in range(1, len(pos_0))]    
#             if max(sum([i>0 for i in diff_pos_0]), sum([i<0 for i in diff_pos_0])) != len(diff_pos_0):
#                  least_condition_met = False    
#         if len(pos_1)>2 :
#             diff_pos_1= [ pos_1[i]-pos_1[i-1] for i in range(1, len(pos_1))]    
#             if max(sum([i>0 for i in diff_pos_1]), sum([i<0 for i in diff_pos_1])) != len(diff_pos_1):
#                  least_condition_met= False    

#         if len(pos_0)==1: # it should be in the middle (quite restirct may not needed)
#             #  ???? we don't knwo the order of chunks when only one is mapped
#             # maybe  we could find by bit flag
#             if pos_0[0] - chunk_size < 0 or  pos_0[0] + 2 * chunk_size > record_length_list[0]:
#                 least_condition_met= False 

#         if len(pos_1)==1: # it should be in the middle (quite restirct may not needed)
#             #  ???? we don't knwo the order of chunks when only one is mapped
#             # maybe  we could find by bit flag
#             if pos_1[0] - chunk_size < 0 or  pos_1[0] + 2 * chunk_size > record_length_list[1]:
#                 #print("case33  ****** ")
#                 #print(chunk_mapped_hap0, pos_0, record_length_list[0],chunk_size) # read_name_chunk_0
#                 #print(chunk_mapped_hap1, pos_1, record_length_list[1])
#                 least_condition_met= False 
#                 # case33  ****** 
#                 # [0, 1, 2] [6895, 3877, 876] 14325 3000
#                 # [3] [7359] 14871
#                 # [0, 1, 2] [6504, 3501, 497] 13155
#                 # [3] [5028] 14881                       

        if not least_condition_met:
            continue
            
        case=0
        condition_needed_met = False
        #### to keep only those with enough space at the end
        ## assumption read_name_chunk_0 and read_name_chunk_0 are in order [0,1,2]
#         if len(pos_0) > 1 and len(pos_1) == 1:
#             if max(pos_1) + 2 * chunk_size < record_length_list[1]:
#                 condition_needed_met = True; case=166
                
        if   len(pos_0) > 1 and len(pos_1) == 1:    #chunk_id (chunk read name): [0,1,2] [3]   or  [1,2] [0]
            if read_name_chunk_0[0] < read_name_chunk_1[0]:        #read_name_chunk_0=[0,1], [2,3]= read_name_chunk_1
                if pos_0[0] < pos_0[1]:   #pos_0=[1000,4000] pos_1[5000]
                    if max(pos_0) + 2*chunk_size < record_length_list[0] and  min(pos_1) - chunk_size > 0:
                        condition_needed_met = True; case=110
                if pos_0[0] > pos_0[1] :   #pos_0=[4000,1000] pos_1[5000]
                    if min(pos_0) - chunk_size > 0  and  min(pos_1) - chunk_size > 0:
                        condition_needed_met = True; case=111
            else: # read_name_chunk_0[0] > read_name_chunk_1[0]:        #read_name_chunk_0=[2,3], [0,1] = read_name_chunk_1
                if pos_0[0] < pos_0[1] :   #pos_0=[1000,4000] pos_1[2000,5000]
                    if min(pos_0) - chunk_size > 0  and  max(pos_1) + 2*chunk_size < record_length_list[1]:
                        condition_needed_met = True; case=112
                if pos_0[0] > pos_0[1] :   #pos_0=[4000,1000] pos_1[5000]
                    if  max(pos_0) + 2*chunk_size < record_length_list[0] and  min(pos_1) - chunk_size > 0:
                        condition_needed_met = True; case=113

        #                     if max(pos_0) + 2 * chunk_size < record_length_list[0]:                        
        #                         condition_needed_met = True; case=11           
#         elif len(pos_1) > 1 and len(pos_0) == 1:
#             if max(pos_1) + 2 * chunk_size < record_length_list[1]:
#                 condition_needed_met = True; case=12
        if   len(pos_1) > 1 and len(pos_0) == 1:    #chunk_id (chunk read name): [0,1,2] [3]   or  [1,2] [0]
            if read_name_chunk_0[0] < read_name_chunk_1[0]:        #read_name_chunk_0=[0,1], [2,3]= read_name_chunk_1
                if pos_1[0] < pos_1[1]:   #pos_0=[1000,4000] pos_1[5000]
                    if max(pos_1) + 2*chunk_size < record_length_list[1] and  min(pos_0) - chunk_size > 0:
                        condition_needed_met = True; case=120
                if pos_1[0] > pos_1[1] :   #pos_0=[4000,1000] pos_1[5000]
                    if min(pos_1) - chunk_size > 0  and  min(pos_0) - chunk_size > 0:
                        condition_needed_met = True; case=121
            else: # read_name_chunk_0[0] > read_name_chunk_1[0]:        #read_name_chunk_0=[2,3], [0,1] = read_name_chunk_1
                if pos_1[0] < pos_1[1] :   #pos_0=[1000,4000] pos_1[2000,5000]
                    if min(pos_1) - chunk_size > 0  and  max(pos_0) + 2*chunk_size < record_length_list[0]:
                        condition_needed_met = True; case=122
                if pos_1[0] > pos_1[1] :   #pos_0=[4000,1000] pos_1[5000]
                    if  max(pos_1) + 2*chunk_size < record_length_list[1] and  min(pos_0) - chunk_size > 0:
                        condition_needed_met = True; case=123

                        
        elif len(pos_1) > 1 and len(pos_0) > 1:
            if 1:
#             if read_name_chunk_0[-1] == read_name_chunk_1[0] or read_name_chunk_0[0] == read_name_chunk_1[-1] :  # [0,1,2]  [2,3]                
#         #                     # [0, 1] [6525, 3581] 13229     [1, 2, 3] [7823, 4825, 579] 9481
#         #                     # [1, 2, 3] [1, 2436, 5443] 13229   [0, 1] [3725, 6731] 14412
#                 condition_needed_met = True; case=623
#                 #print("-", read_name_chunk_0, read_name_chunk_1,case)

#             elif len( set(read_name_chunk_0) & set(read_name_chunk_1) ) == 0 :
#                 # [0, 1, 2] [7200, 4200, 1186] 13229
#                 # [3, 4] [5500, 1027] 14412
# [2, 3] [1161, 4152] 15272
# [0, 1, 2] [7402, 4417, 1438] 13988

                if read_name_chunk_0[0] < read_name_chunk_1[0]:        #read_name_chunk_0=[0,1], [2,3]= read_name_chunk_1
                    if pos_0[0] < pos_0[1] and  pos_1[0] < pos_1[1]:   #pos_0=[1000,4000] pos_1[2000,5000]
                        if max(pos_0) + 2*chunk_size < record_length_list[0] and  min(pos_1) - chunk_size > 0:
                            condition_needed_met = True; case=13
                    if pos_0[0] < pos_0[1] and  pos_1[0] > pos_1[1]:   #pos_0=[1000,4000] pos_1[5000,2000]
                        if max(pos_0) + 2*chunk_size < record_length_list[0] and  max(pos_1) + 2*chunk_size < record_length_list[1]:
                            condition_needed_met = True; case=14
                    if pos_0[0] > pos_0[1] and  pos_1[0] < pos_1[1]:   #pos_0=[4000,1000] pos_1[2000,5000]
                        if min(pos_0) - chunk_size > 0  and  min(pos_1) - chunk_size > 0:
                            condition_needed_met = True; case=15
                    if pos_0[0] > pos_0[1] and  pos_1[0] > pos_1[1]:   #pos_0=[4000,1000] pos_1[5000,2000]
                        if  min(pos_0) - chunk_size > 0 and  max(pos_1) + 2*chunk_size < record_length_list[1]:
                            condition_needed_met = True; case=16

                else: # read_name_chunk_0[0] > read_name_chunk_1[0]:        #read_name_chunk_0=[2,3], [0,1] = read_name_chunk_1
                    if pos_0[0] < pos_0[1] and  pos_1[0] < pos_1[1]:   #pos_0=[1000,4000] pos_1[2000,5000]
                        if min(pos_0) - chunk_size > 0  and  max(pos_1) + 2*chunk_size < record_length_list[1]:
                            condition_needed_met = True; case=17
                    if pos_0[0] < pos_0[1] and  pos_1[0] > pos_1[1]:   #pos_0=[1000,4000] pos_1[5000,2000]
                        if min(pos_0) - chunk_size > 0 and  min(pos_1) - chunk_size > 0:
                            condition_needed_met = True; case=18
                    if pos_0[0] > pos_0[1] and  pos_1[0] < pos_1[1]:   #pos_0=[4000,1000] pos_1[2000,5000]
                        if max(pos_0) + 2*chunk_size < record_length_list[0]  and  max(pos_1) + 2*chunk_size < record_length_list[1]:
                            condition_needed_met = True; case=19
                    if pos_0[0] > pos_0[1] and  pos_1[0] > pos_1[1]:   #pos_0=[4000,1000] pos_1[5000,2000]
                        if  max(pos_0) + 2*chunk_size < record_length_list[0] and  min(pos_1) - chunk_size > 0:
                            print("aaa",pos_0,chunk_size,record_length_list,pos_1)
                            condition_needed_met = True; case=20



        if condition_needed_met : 
            print("*** condition_needed_met ***",case)
            
            print(read_name_chunk_0, pos_0, record_length_list[0]) # read_name_chunk_0
            print(read_name_chunk_1, pos_1, record_length_list[1])
            #print(read_idx, chunk_mapped_hap0, chunk_mapped_hap1) 
            read_name_root =  read_name_root_list[read_idx]
            read_name_chunk = read_name_chunk_list[read_idx]
            read_name_root_1 = read_name_root
            print(read_name_root, ref_names_unique)

            file_input_chop_sam_address_met_idx='.'.join(file_input_chop_sam_address.split(".")[:-1])+"_"+str(read_idx)+".sam"
            ! samtools view -H  {file_input_chop_sam_address} > {file_input_chop_sam_address_met_idx}
            ! wc -l {file_input_chop_sam_address_met_idx}
            ! grep  {read_name_root}  {file_input_chop_sam_address} >> {file_input_chop_sam_address_met_idx}

#             file_input_chop_sam_address_met_idx_bam ='.'.join(file_input_chop_sam_address.split(".")[:-1])+"_.bam" # "+str(read_idx)+"
#             file_input_chop_sam_address_met_idx_bam_sorted ='.'.join(file_input_chop_sam_address.split(".")[:-1])+"_sorted.bam"
#             file_input_chop_sam_address_met_idx_bam_sorted_bai ='.'.join(file_input_chop_sam_address.split(".")[:-1])+"_sorted.bam.bai"
            file_input_chop_sam_address_met_idx_bam ='.'.join(file_input_chop_sam_address_met_idx.split(".")[:-1])+"_.bam" # "+str(read_idx)+"
            file_input_chop_sam_address_met_idx_bam_sorted ='.'.join(file_input_chop_sam_address_met_idx.split(".")[:-1])+"_sorted.bam"
            file_input_chop_sam_address_met_idx_bam_sorted_bai ='.'.join(file_input_chop_sam_address_met_idx.split(".")[:-1])+"_sorted.bam.bai"

            ! samtools view -hb {file_input_chop_sam_address_met_idx} >  {file_input_chop_sam_address_met_idx_bam} 

            ! samtools sort {file_input_chop_sam_address_met_idx_bam} >  {file_input_chop_sam_address_met_idx_bam_sorted} 
            ! samtools index  {file_input_chop_sam_address_met_idx_bam_sorted} 
            # ! ls -atlh {file_input_chop_sam_address_met_idx_bam_sorted}
            ! cp  {file_input_chop_sam_address_met_idx_bam_sorted}  {selected_folder}

            ! cp  {file_input_chop_sam_address_met_idx_bam_sorted_bai}  {selected_folder}

            ! cp {file_input_fa_address}   {selected_folder}
            ! cp {file_input_fa_address+".fai"}   {selected_folder}
            
            
            # we can also keep the original read as fastq
            #file_input_chop_fq_address_met_idx='.'.join(file_input_chop_sam_address.split(".")[:-1])+"_"+str(read_idx)+".sam"

            

            #print(read_idx, read_name_chop) # ref_names
            #                 file_input_chop_part_sam_address_met=file_input_chop_part_sam_address.split(".")[:-1]+"_"+str(read_idx)+".sam"
            #                 ! samtools view -H  {file_input_chop_part_sam_address} > 
            #                 ! grep   {read_name_root}  {file_input_chop_part_sam_address} > 
            print(pos_0, pos_1) # ref_names
            found_any = True
    
    return found_any
   

In [4]:
a=1
a

1

In [ ]:
#estimated_all_F = {1: [17001000, 142538000, 152240000], 2: [90383000], 7: [61163000], 9: [66469000, 68402000], 10: [42538000], 11: [49744000], 16: [21230000], 17: [43682000], 21: [10729000, 11030000], 22: [24372000]}

#estimated_all_M = {2: [90393000, 92284000], 3: [195379000], 4: [70127000], 6: [32529000], 11: [11267000], 12: [80850000], 14: [106795000], 17: [21517000], 21: [9857000, 10714000]}

estimated_all_M = {3: [195379000], 4: [70127000], 6: [32529000], 11: [11267000], 12: [80850000], 14: [106795000], 17: [21517000], 21: [9857000, 10714000]}

#1: [121443000, 142545000],
#  2: [90393000, 92284000],

#estimated_all_M = { 11: [11267000]}


read_eval_code="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/archive/read3b.sh"

threads=2
region_size = 30*1000

working_folder_raw="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/"
working_folder = working_folder_raw+str(int(region_size/1000))+"k/"
! mkdir {working_folder}

   
for chrom in estimated_all_M.keys():
    
    for pos_id in range(len(estimated_all_M[chrom])):
        
        pos= estimated_all_M[chrom][pos_id]
        print(" ******  ---  ***  Working on chr ", chrom, pos)           
        
        folder_name = str(chrom)+"_"+str(pos_id+1)
        print(chrom, folder_name, pos, threads, region_size)
    
    
    
        ###
    
    
        #! bash {read_eval_code}  {chrom} {working_folder+folder_name} {pos} {threads} {region_size} > {working_folder+"log1"}   2>&1 
        print(working_folder+folder_name)


        num_batches_raw = ! ls {working_folder+folder_name} | grep "fai" | wc -l 
        num_batches=int(num_batches_raw[0])

        print(working_folder )
        print("Number of batches is ",num_batches)


        code_chop="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/archive/p4_chop_v1d.py"


        file_input_project_address = working_folder+folder_name+"/"
        
        selected_folder =  working_folder+folder_name+"_selected/"
        print("started for chr"+str(chrom))

        ! mkdir {selected_folder}
        

        #for  id_patch in  range(1,100): #range(1,num_batches+1):    #num_batches+1) : # 10*1000
        for  id_patch in  range(1,num_batches+1):    #num_batches+1) : # 10*1000
            #print("---**---",id_patch)


            #file_input_project_address= "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3a/30k/"+str(chrom)+"_96/"
            #chrom = 2

            #file_input_project_address= "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3a/30k/"+str(chrom)+"_96/"


            file_input_fa_address = file_input_project_address+ "HG004_"+str(chrom)+"_HP1_HP2_"+str(id_patch)+".fa"
            file_input_fai_address =  file_input_project_address+ "HG004_"+str(chrom)+"_HP1_HP2_"+str(id_patch)+".fa.fai"

            (record_name_list, record_length_list)= read_fai(file_input_fai_address)

            file_input_sam_address = file_input_project_address + "HG002_HG004_"+str(chrom)+"_"+str(id_patch)+"_6part.sam" 
            map_qual_tresh = -1
            (read_name_list, num_matched_base_list, ref_name_list, pos_list) = read_sam_cigar(file_input_sam_address, map_qual_tresh)
            #print("there are",len(read_name_list), "child reads. ")


            chunk_size = 3000  # 3kb python code for chopping the child read


            read_name_candidate = find_child_reads_candidate(read_name_list, ref_name_list, num_matched_base_list)

            #print(len(read_name_candidate))


            if len(read_name_candidate): 
                if id_patch %100 ==0: print(id_patch)

                (file_input_chop_sam_address, file_input_chop_part_sam_address) = extract_readname_chop_map(file_input_project_address, chrom, id_patch, code_chop)


                map_qual_tresh = -1
                (read_name_chunk_list, read_name_root_list, ref_name_list, pos_list) = read_sam_chunks(file_input_chop_part_sam_address, map_qual_tresh)

                #print(len(read_name_chunk_list))

                any_found = check_chunks2(read_name_chunk_list, read_name_root_list, ref_name_list, pos_list, file_input_chop_sam_address, file_input_fa_address) 
                if any_found: 
                    a=1
                    #print(any_found,read_name_chunk_list, read_name_root_list )
                    #print(file_input_fa_address)

        print("finsihed for chr"+str(chrom))




mkdir: cannot create directory ‘/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/’: File exists
 ******  ---  ***  Working on chr  3 195379000
3 3_1 195379000 2 30000
/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/3_1
/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/
Number of batches is  7704
started for chr3
300
*** condition_needed_met *** 13
[0, 1, 2] [3, 2943, 5851] 12885
[2, 3] [3007, 6006] 13010
m54334_180925_223328/9962156/ccs ['m64017_191107_163840/90703476/ccs', 'm64017_191112_165230/69796542/ccs']
3 /work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/3_1/HG002_HG004_3_796_chop_0.sam
[3, 2943, 5851] [3007, 6006]
*** condition_needed_met *** 13
[0, 1, 2] [380, 3378, 6282] 12885
[2, 3] [3437, 6437] 13010
m54238_180910_180559/20710207/ccs ['m64017_191107_163840/90703476/ccs', 'm64017_191112_165230/69796542/ccs']
3 /work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/re

In [9]:
a=2
a

2

In [ ]:
#estimated_all_F = {1: [17001000, 142538000, 152240000], 2: [90383000], 7: [61163000], 9: [66469000, 68402000], 10: [42538000], 11: [49744000], 16: [21230000], 17: [43682000], 21: [10729000, 11030000], 22: [24372000]}

#estimated_all_M = {2: [90393000, 92284000], 3: [195379000], 4: [70127000], 6: [32529000], 11: [11267000], 12: [80850000], 14: [106795000], 17: [21517000], 21: [9857000, 10714000]}

estimated_all_M = { 14: [106795000], 17: [21517000], 21: [9857000, 10714000]}

#1: [121443000, 142545000],
#  2: [90393000, 92284000],

#estimated_all_M = { 11: [11267000]}


read_eval_code="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/archive/read3b.sh"

threads=2
region_size = 30*1000

working_folder_raw="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/"
working_folder = working_folder_raw+str(int(region_size/1000))+"k/"
! mkdir {working_folder}

   
for chrom in estimated_all_M.keys():
    
    for pos_id in range(len(estimated_all_M[chrom])):
        
        pos= estimated_all_M[chrom][pos_id]
        print(" ******  ---  ***  Working on chr ", chrom, pos)           
        
        folder_name = str(chrom)+"_"+str(pos_id+1)
        print(chrom, folder_name, pos, threads, region_size)
    
    
    
        ###
    
    
        #! bash {read_eval_code}  {chrom} {working_folder+folder_name} {pos} {threads} {region_size} > {working_folder+"log1"}   2>&1 
        print(working_folder+folder_name)


        num_batches_raw = ! ls {working_folder+folder_name} | grep "fai" | wc -l 
        num_batches=int(num_batches_raw[0])

        print(working_folder )
        print("Number of batches is ",num_batches)


        code_chop="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/archive/p4_chop_v1d.py"


        file_input_project_address = working_folder+folder_name+"/"
        
        selected_folder =  working_folder+folder_name+"_selected/"
        print("started for chr"+str(chrom))

        ! mkdir {selected_folder}
        

        #for  id_patch in  range(1,100): #range(1,num_batches+1):    #num_batches+1) : # 10*1000
        for  id_patch in  range(1,num_batches+1):    #num_batches+1) : # 10*1000
            #print("---**---",id_patch)


            #file_input_project_address= "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3a/30k/"+str(chrom)+"_96/"
            #chrom = 2

            #file_input_project_address= "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3a/30k/"+str(chrom)+"_96/"


            file_input_fa_address = file_input_project_address+ "HG004_"+str(chrom)+"_HP1_HP2_"+str(id_patch)+".fa"
            file_input_fai_address =  file_input_project_address+ "HG004_"+str(chrom)+"_HP1_HP2_"+str(id_patch)+".fa.fai"

            (record_name_list, record_length_list)= read_fai(file_input_fai_address)

            file_input_sam_address = file_input_project_address + "HG002_HG004_"+str(chrom)+"_"+str(id_patch)+"_6part.sam" 
            map_qual_tresh = -1
            (read_name_list, num_matched_base_list, ref_name_list, pos_list) = read_sam_cigar(file_input_sam_address, map_qual_tresh)
            #print("there are",len(read_name_list), "child reads. ")


            chunk_size = 3000  # 3kb python code for chopping the child read


            read_name_candidate = find_child_reads_candidate(read_name_list, ref_name_list, num_matched_base_list)

            #print(len(read_name_candidate))


            if len(read_name_candidate): 
                if id_patch %100 ==0: print(id_patch)

                (file_input_chop_sam_address, file_input_chop_part_sam_address) = extract_readname_chop_map(file_input_project_address, chrom, id_patch, code_chop)


                map_qual_tresh = -1
                (read_name_chunk_list, read_name_root_list, ref_name_list, pos_list) = read_sam_chunks(file_input_chop_part_sam_address, map_qual_tresh)

                #print(len(read_name_chunk_list))

                any_found = check_chunks2(read_name_chunk_list, read_name_root_list, ref_name_list, pos_list, file_input_chop_sam_address, file_input_fa_address) 
                if any_found: 
                    a=1
                    #print(any_found,read_name_chunk_list, read_name_root_list )
                    #print(file_input_fa_address)

        print("finsihed for chr"+str(chrom))




mkdir: cannot create directory ‘/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/’: File exists
 ******  ---  ***  Working on chr  14 106795000
14 14_1 106795000 2 30000
/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/14_1
/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/
Number of batches is  1088
started for chr14
mkdir: cannot create directory ‘/work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/14_1_selected/’: File exists
200
300
*** condition_needed_met *** 17
[2, 3] [9851, 12852] 20980
[0, 1, 2] [658, 3659, 6660] 13205
m54238_180915_185611/48824514/ccs ['m64017_191112_165230/170459530/ccs', 'm64017_191113_230503/48103531/ccs']
3 /work/FAC/FBM/DBC/cdessim2/default/smajidi1/recombination/read_read_v3b/30k/14_1/HG002_HG004_14_306_chop_15.sam
[9851, 12852] [658, 3659, 6660]
*** condition_needed_met *** 17
[2, 3] [9851, 12852] 20980
[0, 1, 2] [3007, 6009, 9008] 15379
m54238_18091

In [ ]:
# 1_Mo: [121443000, 142545000] [28, 14]
# 1_Fa: [17001000, 142538000] [51, 50]
# 2_Mo: [90393000, 92284000] [70, 23]
# 2_Fa: [90383000, 92312000] [58, 15]
# 3_Mo: [195379000, 195699000] [24, 13]
# 3_Fa: [147605000, 195494000] [5, 9]
# 4_Mo: [49289000, 70127000] [12, 16]
# 4_Fa: [49142000, 132664000] [15, 9]
# 5_Mo: [64465000, 177221000] [8, 7]
# 5_Fa: [49421000, 180891000] [5, 15]
# 6_Mo: [32529000, 57518000] [20, 12]
# 6_Fa: [29917000, 103741000] [9, 10]
# 7_Mo: [56431000, 62747000] [5, 9]
# 7_Fa: [61163000, 158338000] [41, 15]
# 8_Mo: [2233000, 12446000] [5, 5]
# 8_Fa: [2212000, 143094000] [12, 7]
# 9_Mo: [124659000, 141121000] [6, 7]
# 9_Fa: [66469000, 68402000] [30, 31]
# 10_Mo: [42368000, 47028000] [12, 7]
# 10_Fa: [42538000, 51825000] [35, 12]
# 11_Mo: [11267000, 51584000] [14, 9]
# 11_Fa: [48884000, 49744000] [15, 17]
# 12_Mo: [34834000, 80850000] [9, 15]
# 12_Fa: [38364000, 80847000] [7, 6]
# 13_Mo: [50459000, 75383000] [9, 4]
# 13_Fa: [112946000, 114910000] [9, 4]
# 14_Mo: [101373000, 106795000] [12, 18]
# 14_Fa: [24437000, 106802000] [4, 8]
# 15_Mo: [22827000, 34672000] [9, 12]
# 15_Fa: [22513000, 25119000] [7, 6]
# 16_Mo: [26386000, 46402000] [7, 248]
# 16_Fa: [21230000, 46405000] [17, 61]
# 17_Mo: [21517000, 21904000] [14, 11]
# 17_Fa: [43682000, 44336000] [18, 11]
# 18_Mo: [47286000, 77367000] [4, 4]
# 18_Fa: [70637000, 76778000] [6, 4]
# 19_Mo: [16041000, 43740000] [7, 10]
# 19_Fa: [39270000, 48424000] [15, 10]
# 20_Mo: [5724000, 62917000] [10, 6]
# 20_Fa: [26304000, 56764000] [4, 2]
# 21_Mo: [9857000, 10714000] [17, 21]
# 21_Fa: [10729000, 11030000] [71, 18]
# 22_Mo: [18837000, 51227000] [3, 8]
# 22_Fa: [16925000, 24372000] [10, 18]

In [ ]:
# 1_Mo: 121443000 28
# 1_Fa: 17001000 51
# 2_Mo: 92284000 23
# 2_Fa: 90383000 58
# 3_Mo: 195379000 24
# 3_Fa: 195494000 9
# 4_Mo: 70127000 16
# 4_Fa: 49142000 15
# 5_Mo: 64465000 8
# 5_Fa: 180891000 15
# 6_Mo: 32529000 20
# 6_Fa: 103741000 10
# 7_Mo: 62747000 9
# 7_Fa: 61163000 41
# 8_Mo: 2233000 5
# 8_Fa: 2212000 12
# 9_Mo: 141121000 7
# 9_Fa: 68402000 31
# 10_Mo: 42368000 12
# 10_Fa: 42538000 35
# 11_Mo: 11267000 14
# 11_Fa: 49744000 17
# 12_Mo: 80850000 15
# 12_Fa: 38364000 7
# 13_Mo: 50459000 9
# 13_Fa: 112946000 9
# 14_Mo: 106795000 18
# 14_Fa: 106802000 8
# 15_Mo: 34672000 12
# 15_Fa: 22513000 7
# 16_Mo: 26386000 7
# 16_Fa: 21230000 17
# 17_Mo: 21517000 14
# 17_Fa: 43682000 18
# 18_Mo: 47286000 4
# 18_Fa: 70637000 6
# 19_Mo: 43740000 10
# 19_Fa: 39270000 15
# 20_Mo: 5724000 10
# 20_Fa: 26304000 4
# 21_Mo: 10714000 21
# 21_Fa: 11030000 18
# 22_Mo: 51227000 8
# 22_Fa: 24372000 18
